In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/customized-knowledge-qa
!pip install farm-haystack[colab,inference]

# **Initializing the DocumentStore**

Use TextIndexingPipeline to convert the files into Haystack Document objects and write them into the DocumentStore

In [2]:
import os
from haystack.document_stores import InMemoryDocumentStore
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

doc_dir = 'knowledge/data'
files_to_index = [doc_dir + '/' + f for f in os.listdir(doc_dir)]
document_store = InMemoryDocumentStore(use_bm25=True)
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Updating BM25 representation...: 100%|██████████| 59/59 [00:00<00:00, 8219.21 docs/s]


{'documents': [<Document: {'content': '===AST – BẮT ĐẦU MỘT CHƯƠNG MỚI===\n\n==KQKD Q4/22: HỒI PHỤC MẠNH MẼ NHỜ DÒNG KHÁCH QUỐC TẾ==\nDoanh thu Q4/22 tăng 674% nhờ sản lượng khách thông qua các cảng hàng không tăng 534% svck trong đó (1) khách nội địa tăng 407% svck, và (2) khách quốc tế tăng 60 lần svck, giúp LN ròng Q4/22 đạt 18 tỷ sv lỗ ròng 16 tỷ trong Q4/21, khiến LN ròng 2022 đạt 23,1 tỷ qua đó AST vẫn đủ điều kiện để tiếp tục niêm yết trên HSX. Hơn nữa, so sánh với quý trước, mặc dù sản lượng khách nội địa Q4/22 giảm 36,4% sv quý trước, sản lượng khách quốc tế Q4/22 tăng 8,2% sv quý trước, giúp doanh thu/lợi nhuận tăng 16,8%/39,6% sv quý trước, cho thấy kết quả kinh doanh của AST có độ tương quan cao với sự phục hồi của khách quốc tế.\n\n==TRIỂN VỌNG 2023-24: BƯỚC VÀO GIAI ĐOẠN PHỤC HỒI VÀ TĂNG TRƯỞNG MẠNH MẼ==\nMới đây, Việt Nam đã được bổ sung vào danh sách các quốc gia thí điểm cho du lịch theo đoàn của Trung Quốc bắt đầu từ 15/03/2023. ', 'content_type': 'text', 'score': Non

# **Initializing the Retriever**

Retriever sifts through all the Documents and returns only the ones relevant to the question (use BM25 algorithm in this demonstration)

In [3]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

# **Initializing the Reader**

Reader scans the texts it received from the Retriever and extracts the top answer candidates

In [4]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='dantern/xlm-roberta-base-vn-dplat', use_gpu=True)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# **Creating the Retriever-Reader Pipeline**

In [5]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

# **Asking a Question**

You can set the number of documents you want the Reader and Retriever to return using the top-k parameter

In [6]:
prediction = pipe.run(
    query='Lợi nhuận ròng của FCN năm 2022 là bao nhiêu?', params={'Retriever': {'top_k': 10}, 'Reader': {'top_k': 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:18<00:00, 18.73s/ Batches]


In [7]:
prediction

{'query': 'Lợi nhuận ròng của FCN năm 2022 là bao nhiêu?',
 'no_ans_gap': 9.494028687477112,
 'answers': [<Answer {'answer': '43,0 nghìn tỷ đồng (+9% svck) và 1.530 tỷ đồng (-19% svck),', 'type': 'extractive', 'score': 0.8212946057319641, 'context': 'ơn dự kiến. Cả năm 2022, phí BH gốc và LN ròng đạt 43,0 nghìn tỷ đồng (+9% svck) và 1.530 tỷ đồng (-19% svck), hoàn thành lần lượt 99%/93% dự phóng.\n\n', 'offsets_in_document': [{'start': 606, 'end': 665}], 'offsets_in_context': [{'start': 51, 'end': 110}], 'document_ids': ['f3170f5d0ad2cb63a6ce3f4f058b178d'], 'meta': {'_split_id': 0}}>,
  <Answer {'answer': '342 tỷ đồng (-37% svck).', 'type': 'extractive', 'score': 0.7450861930847168, 'context': ' đông tập đoàn trong Q4/22 là 11,2 nghìn tỷ đồng (+2% svck) và 342 tỷ đồng (-37% svck). LN ròng giảm chủ yếu do lỗ từ HĐKD BH tăng 28% svck, được bù đ', 'offsets_in_document': [{'start': 233, 'end': 257}], 'offsets_in_context': [{'start': 63, 'end': 87}], 'document_ids': ['f3170f5d0ad2cb63a6ce3f

In [8]:
from haystack.utils import print_answers

print_answers(prediction, details='minimum')  ## Choose from `minimum`, `medium`, and `all`

'Query: Lợi nhuận ròng của FCN năm 2022 là bao nhiêu?'
'Answers:'
[   {   'answer': '43,0 nghìn tỷ đồng (+9% svck) và 1.530 tỷ đồng (-19% svck),',
        'context': 'ơn dự kiến. Cả năm 2022, phí BH gốc và LN ròng đạt 43,0 '
                   'nghìn tỷ đồng (+9% svck) và 1.530 tỷ đồng (-19% svck), '
                   'hoàn thành lần lượt 99%/93% dự phóng.\n'
                   '\n'},
    {   'answer': '342 tỷ đồng (-37% svck).',
        'context': ' đông tập đoàn trong Q4/22 là 11,2 nghìn tỷ đồng (+2% '
                   'svck) và 342 tỷ đồng (-37% svck). LN ròng giảm chủ yếu do '
                   'lỗ từ HĐKD BH tăng 28% svck, được bù đ'},
    {   'answer': '39 tỷ đồng',
        'context': 'vck xuống 3.045 tỷ đồng, trong khi LN ròng giảm mạnh 42,8% '
                   'svck xuống 39 tỷ đồng, chỉ hoàn thành 36,6% dự phóng cả '
                   'năm của chúng tôi do: (1) Biên LN g'},
    {   'answer': 'giảm 16,3% svck',
        'context': 'ong năm 2022, doanh thu của MWG tăng trư

In [9]:
prediction = pipe.run(
    query='Nguyên nhân dẫn đến biên lợi nhuận gộp của C4G tại Q4/22 giảm mạnh ?', params={'Retriever': {'top_k': 10}, 'Reader': {'top_k': 5}}
)
print_answers(prediction, details='minimum')

Inferencing Samples: 100%|██████████| 1/1 [00:18<00:00, 18.41s/ Batches]

'Query: Nguyên nhân dẫn đến biên lợi nhuận gộp của C4G tại Q4/22 giảm mạnh ?'
'Answers:'
[   {   'answer': 'crack spread xăng giảm (crack spread xăng tại châu Á Q4/22: '
                  '7,6 USD/thùng; -51% svck) và tỷ suất LN hóa dầu thấp hơn',
        'context': '8 điểm % svck do crack spread xăng giảm (crack spread xăng '
                   'tại châu Á Q4/22: 7,6 USD/thùng; -51% svck) và tỷ suất LN '
                   'hóa dầu thấp hơn. Trong Q4/22, BS'},
    {   'answer': 'do giá nguyên vật liệu xây dựng tăng mạnh đầu năm 2022 dẫn '
                  'đến biên LN gộp của mảng Xây dựng giảm 6,9 điểm % svck',
        'context': 'nh 8,9 điểm % svck do giá nguyên vật liệu xây dựng tăng '
                   'mạnh đầu năm 2022 dẫn đến biên LN gộp của mảng Xây dựng '
                   'giảm 6,9 điểm % svck. Kết quả, LN ròng'},
    {   'answer': 'các dự án ghi nhận trong kỳ có biên lợi nhuận thấp (thông '
                  'tin chi tiết dự án không được doanh nghiệp tiết lộ',
        '